In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [9]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42.0F"

In [20]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

### Custom Tool

In [22]:
import requests
# Note: Only v1 of pydantic is supported at the time of this writing
# Please see this issue for more details: https://github.com/langchain-ai/langchain/issues/9441
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [23]:
get_current_temperature.name

'get_current_temperature'

In [24]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [25]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [26]:
from langchain.tools.render import format_tool_to_openai_function

In [28]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 13.8°C'

In [30]:
# !pip install wikipedia
import wikipedia

In [31]:
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [34]:
print(search_wikipedia.name, search_wikipedia.args, search_wikipedia.description)

search_wikipedia {'query': {'title': 'Query', 'type': 'string'}} search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.


In [35]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [36]:
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: OpenAI\nSummary: OpenAI is a U.S. artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the declared intention of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".\nAs one of the leading organizations of the AI Spring, it has developed several large language models, advanced image generation models, and previously, also open-source models. Its release of ChatGPT has been credited with starting the artificial intelligence spring.The organization consists of the n

In [37]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

### Routing

In [44]:
from langchain.chat_models import ChatOpenAI

functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [45]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

In [46]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "langchain"\n}', 'name': 'search_wikipedia'}})

In [47]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [48]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}', 'name': 'get_current_temperature'}})

In [49]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [50]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [51]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [52]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [53]:
result.tool

'get_current_temperature'

In [54]:
get_current_temperature(result.tool_input)

'The current temperature is 11.2°C'

In [55]:
result = chain.invoke({"input": "hi!"})

In [56]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

In [57]:
type(result)

langchain_core.agents.AgentFinish

In [58]:
# If a function is called -> AgentAction
# If no function is called -> AgentFinish

from langchain.schema.agent import AgentFinish

def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [59]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [62]:
result = chain.invoke({"input": "What is the weather in Denver right now?"})

In [63]:
result

'The current temperature is 1.7°C'